# Transform data

In [1]:

import pandas as pd

# Načti vstupní soubor
input_excel = "receptury-po-technologiich_final.xlsx"
xlsx = pd.ExcelFile(input_excel)
sheet_names = xlsx.sheet_names

# Načti vany
df_linka = xlsx.parse("vany_a_vzdalenosti")

# Získání všech recepturových listů
recept_sheets = [s for s in sheet_names if s != "vany_a_vzdalenosti"]

# Vytvoření výstupního seznamu řádků
recept_rows = []

for idx, sheet in enumerate(recept_sheets):
    df_recept = xlsx.parse(sheet)
    tech_id = f"tech{idx + 1}"  # tech1, tech2, ...

    if "nazev_operace" not in df_recept.columns:
        print(f"List '{sheet}' neobsahuje sloupec 'nazev_operace'.")
        print(f"Dostupné sloupce: {list(df_recept.columns)}")
        continue

    df_recept = df_recept.reset_index(drop=True)

    for i, row in df_recept.iterrows():
        nazev = row["nazev_operace"]
        odpovida = df_linka[df_linka["typ vany"] == nazev]

        if odpovida.empty:
            print(f"Varování: V receptuře '{sheet}' není nalezena vana pro operaci '{nazev}'")
            continue

        id_vany = odpovida["id_vany"].values[0]

        recept_rows.append({
            "tech": tech_id,
            "id_vany": id_vany,
            "poradi_operace": i + 1,
            "cas_min": int(row["cas_min"]),
            "cas_opt": int(row["cas_opt"]),
            "cas_max": int(row["cas_max"]),
            "okap": 1 if int(row["okap_cas"]) > 0 else 0,
            "okap_cas": int(row["okap_cas"])
        })

# Sestavení DataFrame a export do Excelu
df_recept = pd.DataFrame(recept_rows)
with pd.ExcelWriter("data1.xlsx") as writer:
    df_linka.to_excel(writer, sheet_name="linka", index=False)
    df_recept.to_excel(writer, sheet_name="recept", index=False)

print("Soubor 'data1.xlsx' byl úspěšně vytvořen.")


Varování: V receptuře 'Tech_1_-_FE_plus_moř' není nalezena vana pro operaci 'Demi '
Varování: V receptuře 'Tech_2_-_Fe_bez_moř' není nalezena vana pro operaci 'Demi '
Varování: V receptuře 'Tech_3_-_Zn_plus_moř' není nalezena vana pro operaci 'Demi '
Soubor 'data1.xlsx' byl úspěšně vytvořen.
